In [52]:
import math

# Performance Modelling for CUDNN on Titan V for GEMM Workloads

## Layer Class Definitions
First, let's define a few classes for convenience. Since we're focusing on modeling Convolutional and Gemm layers, we'll define a class for each.

In [53]:
# The ConvLayer represents a convolution with the following parameters:
#     The input consists of Ni input channels each of which consist of Ny x Nx images. The shape should be (Ni, Nx, Ny)
#     One convolution filter has dimensions Kx x Ky x Ni. The input filter_shape should be (Kx, Ky) and the Ni dimension is assumed.
class ConvLayer:
  def __init__(self, name, input_shape, filter_shape, num_filters, batch_size = 1, padding = 0, stride = 1):
    self.name = name
    self.type = "CONV"
    self.input_shape = input_shape      #(Ni, Nx, Ny)
    self.filter_shape = filter_shape    #(Kx, Ky)
    self.stride = stride
    self.batch_size = batch_size
    self.padding = padding
    self.num_filters = num_filters

# The GemmLayer represents a 2D matrix multiply
#     The first matrix has dimensions m x k and the second matrix has dimensions k x n.
#     This gives an output with dimensions m x n.
class GemmLayer:
  def __init__(self, name, m, k, n, batch_size = 1):
    self.name = name
    self.type = "GEMM"
    self.m = m
    self.k = k
    self.n = n
    self.batch_size = batch_size

## Titan V Parameters, Constants, and Test Model Parameters

- Defined here are some performance and hardware stats for the Titan V GPU
- We also define several NN models from sequences of convolutional/GEMM layers.

In [68]:
# Define parameters or lists of parameters here

#============================================
# From https://www.anandtech.com/show/12673/titan-v-deep-learning-deep-dive and https://www.nvidia.com/en-us/titan/titan-v/
# Parameters from NVIDIA Titan V
num_CUDA_Cores = 5120     # Total number of CUDA cores
num_Tensor_Cores = 640    # Tensor Cores, Hopefully not used
num_ROPs = 96             # Render Output Units, Hopefully not used
num_Streaming_Multiprocessors = 80    # This is different from the number in the slides.

Core_Clock = 1200000000             # 1200MHz
Boost_Clock = 1455000000            # 1455MHz
Memory_Clock = 850000000            # 850MHz
Memory_Data_Rate = 1825361101           # 1.7Gbps HBM2 (bits per second not bytes)

Memory_Bus_Width = 3072             # 3072 bits
Memory_Bandwidth = 701153411072     # 653GB/sec in Bytes

# bandwidth from L2 to L1
# calculated by numbers in table 2 of this link: https://arxiv.org/pdf/1810.07269.pdf
# L2 to L1 has 128B line, 32 ways
# 128 byte 32 lines = 4096 bytes/cycle
# 4096 bytes/cycle * Boost_Clock = 5.96e12 Bytes/sec  
L2_to_L1_Bandwidth = 5.96e12     # 5.9 TB/sec

VRAM_size = 12000000000             # 12GB
L2_size = 4500000                   # 4.5MB
Shared_Memory_Size = 98304          # 96KB

Single_Precision_Performance = 13800000000000   # 13.8 TFLOPS
Double_Precision_Performance =  6900000000000   # 6.9 TFLOPS
Half_Precision_Performance =   27600000000000   # 27.6 TFLOPS
Integer_INT8_Performance =     55200000000000   # 55.2 TOPS
Tensor_Performance =          110000000000000   # 110 TFLOPS

# Memory access latency values:
# source: https://arxiv.org/pdf/1810.07269.pdf
# https://arxiv.org/pdf/1804.06826.pdf
L1_latency = 19   #cycles, instead of 28 from the first link, 19 from second for shared latency
L2_latency = 198  #cycles, from second link
VRAM_latency = 100 #nanoseconds

# Main memory bandwidth, commented out to check
main_memory_bandwidth = 126e9 #15754000000 # 15.754 GB/sec in Bytes (decimal)


# ran an empty/arbitrary kernel on the TitanV and took the minimum execution time with nvprof
STARTUP_TIME = 1.216e-6      # seconds

#============================================

# We are making the following simplifying assumptions:

#   - We are assuming always using the boost clock
#   - Since we don't know which clock was used to computer performance numbers, we are basing CPOP of info from https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html?fbclid=IwAR3_uznGWPcbbWDJMv2Gsa1VgqOCpfrBMoK7yJdhrIvvjz7DumnH4XDzlbY#arithmetic-instructions
# def compute_CPOP(): #precision):
#   #TODO: Finish
#   return 1
# number_of_cycles_per_op = compute_CPOP()

# The layer specifications we tested in Mini-Project 1 are as follows:
# Conv1: Nx=224 Ny=224 Kx=3 Ky=3 Ni=64 Nn=64
# Conv2: Nx=14 Ny=14 Kx=3 Ky=3 Ni=512 Nn=512
# Class1: Ni=25088 Nn=4096
# Class2: Ni=4096 Nn=1024

# The "precision" variable can have one of the following string values:
# "FP16" - Half precision 16-bit floating point
# "FP32" - Single precision 32-bit floating point
# "FP64" - Double precision 64-bit floating point
# "INT8" - 8-bit integer
def verifyPrecisionValue(precision):
  if (precision != "FP16" and precision != "FP32" and precision != "FP64" and precision != "INT8"):
    return False
  else:
    return True

#============================================
# Input Parameters Here
conv_layers = []

# Conv1 layers with multiple batch sizes.
conv_layers.append(ConvLayer("Conv_1_b1", (64,224,224), (3,3), 64, 1))
conv_layers.append(ConvLayer("Conv_1_b2", (64,224,224), (3,3), 64, 2))
conv_layers.append(ConvLayer("Conv_1_b4", (64,224,224), (3,3), 64, 4))
conv_layers.append(ConvLayer("Conv_1_b8", (64,224,224), (3,3), 64, 8))
conv_layers.append(ConvLayer("Conv_1_b16", (64,224,224), (3,3), 64, 16))
conv_layers.append(ConvLayer("Conv_1_b32", (64,224,224), (3,3), 64, 32))
conv_layers.append(ConvLayer("Conv_1_b64", (64,224,224), (3,3), 64, 64))
conv_layers.append(ConvLayer("Conv_1_b128", (64,224,224), (3,3), 64, 128))

# Conv2 layers with multiple batch sizes.
conv_layers.append(ConvLayer("Conv_2_b1", (512,14,14), (3,3), 512, 1))
conv_layers.append(ConvLayer("Conv_2_b2", (512,14,14), (3,3), 512, 2))
conv_layers.append(ConvLayer("Conv_2_b4", (512,14,14), (3,3), 512, 4))
conv_layers.append(ConvLayer("Conv_2_b8", (512,14,14), (3,3), 512, 8))
conv_layers.append(ConvLayer("Conv_2_b16", (512,14,14), (3,3), 512, 16))
conv_layers.append(ConvLayer("Conv_2_b32", (512,14,14), (3,3), 512, 32))
conv_layers.append(ConvLayer("Conv_2_b64", (512,14,14), (3,3), 512, 64))
conv_layers.append(ConvLayer("Conv_2_b128", (512,14,14), (3,3), 512, 128))

gemm_layers = []
# Note: The following specs are not for FC layers.
# FC layers use a series of dot products, not a full matrix multiply.
# Using Gemm with im2col can be used to compute convolution layers.
gemm_layers.append(GemmLayer("Gemm_0", 25088, 25088, 4096, 1))
gemm_layers.append(GemmLayer("Gemm_1", 4096, 4096, 1024, 1))
#============================================

## Functions to help calculate execution time

In [55]:
# assume a slightly higher-end computer setup
# DRAM clocked at 3600 MHz, CAS 16 latency, which has first word latency of 8.89 ns
# know that Titan V interfaces with PCIe 3.0x16, so 32 GB/s; 
# PCIe 3.0 bus latency - https://www.dolphinics.com/download/WHITEPAPERS/Dolphinproductbrochure.pdf
# approximately 500 ns
def latency_of(source, clock_rate):
  """
  :param source: string denoting whether we're leaving L1/shared, L2, or VRAM
  :return: float value for latency penalty
  """
  if source in ("L1", "shared"):
    return L1_latency / clock_rate
  elif source == "L2":
    return L2_latency / clock_rate
  elif source == "VRAM":
    return VRAM_latency
  else:
    # copying from CPU DRAM to GPU VRAM
    return 5e-7 + 8.89e-9

In [56]:
# time to load FROM main memory (CPU) TO GPU VRAM
def time_to_load_from_main_memory(precision, size_data):
  load_time = 0
  if precision == "FP16":
    load_time = size_data*2/main_memory_bandwidth + latency_of("main", Boost_Clock)
  elif precision == "FP32":
    load_time = size_data*4/main_memory_bandwidth + latency_of("main", Boost_Clock)
  elif precision == "FP64":
    load_time = size_data*8/main_memory_bandwidth + latency_of("main", Boost_Clock)
  elif precision == "INT8":
    load_time = size_data*1/main_memory_bandwidth + latency_of("main", Boost_Clock)
  else:
    print("ERROR: Invalid precision")
  return load_time

# time to load FROM VRAM to L2
def time_to_load_from_VRAM(precision, size_data): 
  load_time = 0
  if precision == "FP16":
    load_time = size_data*2/Memory_Bandwidth + latency_of("VRAM", Boost_Clock)
  elif precision == "FP32":
    load_time = size_data*4/Memory_Bandwidth + latency_of("VRAM", Boost_Clock)
  elif precision == "FP64":
    load_time = size_data*8/Memory_Bandwidth + latency_of("VRAM", Boost_Clock)
  elif precision == "INT8":
    load_time = size_data*1/Memory_Bandwidth + latency_of("VRAM", Boost_Clock)
  else:
    print("ERROR: Invalid precision")
  return load_time

# time to load from L2 to L1 WITHOUT latency penalty
# this is different from the following function, and will be used to help compute tiling memory costs
def time_to_load_from_L2_to_L1(precision, size_data):
  load_time = 0
  if precision == "FP16":
    load_time = size_data*2/L2_to_L1_Bandwidth 
  elif precision == "FP32":
    load_time = size_data*4/L2_to_L1_Bandwidth 
  elif precision == "FP64":
    load_time = size_data*8/L2_to_L1_Bandwidth 
  elif precision == "INT8":
    load_time = size_data*1/L2_to_L1_Bandwidth 
  else:
    print("ERROR: Invalid precision")
  return load_time

# time to load FROM L2 TO L1
def time_to_load_from_device_memory(precision, size_data): #size_data is given in terms of values, not taking precision into account.
  load_time = 0
  if precision == "FP16":
    load_time = size_data*2/L2_to_L1_Bandwidth + latency_of("L2", Boost_Clock)
  elif precision == "FP32":
    load_time = size_data*4/L2_to_L1_Bandwidth + latency_of("L2", Boost_Clock)
  elif precision == "FP64":
    load_time = size_data*8/L2_to_L1_Bandwidth + latency_of("L2", Boost_Clock)
  elif precision == "INT8":
    load_time = size_data*1/L2_to_L1_Bandwidth + latency_of("L2", Boost_Clock)
  else:
    print("ERROR: Invalid precision")
  return load_time

# time to load FROM L1/Shared TO kernelexecution/processors
# shared memory is same as L1 cache in terms of speed 
def time_to_load_from_shared_memory(precision, size_data): #size_data is given in terms of values, not taking precision into account.
  load_time = 0
  if precision == "FP16":
    load_time = size_data*2/Memory_Bandwidth + latency_of("L1", Boost_Clock)
  elif precision == "FP32":
    load_time = size_data*4/Memory_Bandwidth + latency_of("L1", Boost_Clock)
  elif precision == "FP64":
    load_time = size_data*8/Memory_Bandwidth + latency_of("L1", Boost_Clock)
  elif precision == "INT8":
    load_time = size_data*1/Memory_Bandwidth + latency_of("L1", Boost_Clock)
  else:
    print("ERROR: Invalid precision")
  return load_time
  
# empirical meaning using the TFLOPs data from cell 2 instead of the NVIDIA CPOP table link
def time_to_execute(precision, num_ops, empirical = True): 
  if not empirical:  # using NVIDIA table link 
    if precision == "FP16":
      execution_time = num_ops / (128 * num_Streaming_Multiprocessors * Boost_Clock)  
    elif precision == "FP32":
      execution_time = num_ops / (64 * num_Streaming_Multiprocessors * Boost_Clock)  
    elif precision == "FP64":
      execution_time = num_ops / (32 * num_Streaming_Multiprocessors * Boost_Clock)  
    elif precision == "INT8":
      # not much in documentation on the throughput for INT8 but links such as: 
      # https://forums.developer.nvidia.com/t/titan-v-white-paper-and-int8-tops/72776
      # https://docs.nvidia.com/cuda/pascal-tuning-guide/index.html
      # imply that INT8 throughput is 4x that of FP32 
      execution_time = num_ops / (4 * 64 * num_Streaming_Multiprocessors * Boost_Clock)  
    else:
      print("ERROR: Invalid precision")
      return -1
  else:
    if precision == "FP16":
      execution_time = num_ops / Half_Precision_Performance
    elif precision == "FP32":
      execution_time = num_ops / Single_Precision_Performance
    elif precision == "FP64":
      execution_time = num_ops / Double_Precision_Performance
    elif precision == "INT8":
      execution_time = num_ops / Integer_INT8_Performance
    else:
      print("ERROR: Invalid precision")
      return -1
  return execution_time

In [57]:
# In the project spec, the professor mentioned finding the max time of each of compute, memory, and buffer bandwidths.
# We are supposed to be better than this, but this seems doable.


def find_compute_bound_time(precision, layer_shape, empirical = True):
  '''
  :param precision: which precision type, FP16, FP32, ...
  :type precision: string
  :param layer_shape: a Layer object denoting layer properties
  :type layer_shape: Layer
  :param empirical: choose whether to use "empirical" data from cell 2 or "ideal" from table linked
  :type empirical: bool

  :return: float denoting expected compute time in seconds

  '''
  # Based on Number of Operations
  expected_time = 0
  if (layer_shape.type == "GEMM"):
    num_mults = layer_shape.m * layer_shape.k * layer_shape.n
    num_adds = layer_shape.m * (layer_shape.k - 1) * layer_shape.n
    total_ops = num_mults + num_adds
    expected_time = time_to_execute(precision, total_ops, empirical)

  elif (layer_shape.type == "CONV"):
    # Number of dot product ops:  ((nx -kx + 2P)/S + 1) * ((ny - ky + 2P)/S + 1) * Nn 
    # number of multiply and add ops per stride: (kx * ky * Ni) multiplies and (kx * ky * Ni - 1) adds
    num_dot_prods = (math.floor((layer_shape.input_shape[1] - layer_shape.filter_shape[0] + 2*layer_shape.padding) / layer_shape.stride) + 1) \
                        * (math.floor((layer_shape.input_shape[2] - layer_shape.filter_shape[1] + 2*layer_shape.padding) / layer_shape.stride) + 1) \
                        * layer_shape.num_filters
    num_ops_per_dot = 2 * (layer_shape.filter_shape[0] * layer_shape.filter_shape[1] * layer_shape.input_shape[0])
    total_ops = num_dot_prods * num_ops_per_dot * layer_shape.batch_size
    expected_time = time_to_execute(precision, total_ops, empirical)
  expected_time = expected_time
  return expected_time

# #test code
# compute_bound_time = find_compute_bound_time("FP16", ConvLayer("Conv_1", (64,224,224),(3,3), 1, 0, 1), False)
# print("Table: " + str(compute_bound_time) + " seconds")
# compute_bound_time = find_compute_bound_time("FP16", ConvLayer("Conv_1", (64,224,224),(3,3), 1, 0, 1), True)
# print("Empirical: " + str(compute_bound_time) + " seconds")


def find_memory_bound_time(precision,layer_shape):
  # From Host
  expected_time = 0
  if (layer_shape.type == "GEMM"):
    # This is the bare minimum assuming each value is loaded exactly once.
    size_load_data = layer_shape.k * (layer_shape.m + layer_shape.n * layer_shape.batch_size)
    size_out_data = layer_shape.m * layer_shape.n * layer_shape.batch_size
  elif (layer_shape.type == "CONV"):
    # This is the bare minimum assuming each value is loaded exactly once.
    size_load_data = layer_shape.input_shape[0]*(layer_shape.input_shape[1]*layer_shape.input_shape[2]*layer_shape.batch_size + layer_shape.num_filters*layer_shape.filter_shape[0]*layer_shape.filter_shape[1])
    size_out_data = layer_shape.num_filters*(layer_shape.input_shape[1]-layer_shape.filter_shape[0]+1)*(layer_shape.input_shape[2]-layer_shape.filter_shape[1]+1)*layer_shape.batch_size
  print("Data size is load "+str(size_load_data)+" and output "+str(size_out_data))
  expected_time = time_to_load_from_main_memory(precision,size_load_data+size_out_data)
  return expected_time

def find_buffer_bandwidth_bound_time(precision,layer_shape):
  # DRAM Staging Buffer Bandwidth on Device
  expected_time = 0
  if (layer_shape.type == "GEMM"):
    # This is the bare minimum assuming each value is loaded exactly once.
    size_load_data = layer_shape.k * (layer_shape.m + layer_shape.n * layer_shape.batch_size)
    size_out_data = layer_shape.m * layer_shape.n * layer_shape.batch_size
  elif (layer_shape.type == "CONV"):
    # This is the bare minimum assuming each value is loaded exactly once.
    size_load_data = layer_shape.input_shape[0]*(layer_shape.input_shape[1]*layer_shape.input_shape[2]*layer_shape.batch_size + layer_shape.num_filters*layer_shape.filter_shape[0]*layer_shape.filter_shape[1])
    size_out_data = layer_shape.num_filters*(layer_shape.input_shape[1]-layer_shape.filter_shape[0]+1)*(layer_shape.input_shape[2]-layer_shape.filter_shape[1]+1)*layer_shape.batch_size
  print("Data size is load "+str(size_load_data)+" and output "+str(size_out_data))
  expected_time = time_to_load_from_L2_to_L1(precision,size_load_data+size_out_data)
  return expected_time

## Roofline Model

In [58]:
def roofline_model(precision,layer_shape):
  """ 
  Returns a tuple of format(execution_time, [compute, memory, buffer])
  """
  execution_time = 0

  compute_bound_time = find_compute_bound_time(precision,layer_shape)
  print("Compute: "+str(compute_bound_time))
  memory_bound_time = find_memory_bound_time(precision,layer_shape)
  print("Memory: "+str(memory_bound_time))
  buffer_bound_time = find_buffer_bandwidth_bound_time(precision,layer_shape)
  print("Buffer: "+str(buffer_bound_time))

  execution_time = compute_bound_time

  if (memory_bound_time > execution_time and memory_bound_time > buffer_bound_time):
    execution_time = memory_bound_time
    return (execution_time, "memory")
  if (buffer_bound_time > execution_time and buffer_bound_time > memory_bound_time):
    execution_time = buffer_bound_time
    return (execution_time, "buffer")
  return (execution_time+STARTUP_TIME, "compute")


In [59]:
def gemm(precision,gemm_layer):   # Roofline Model Baseline
  execution_time = 0

  # Here is some more filler code

  results = roofline_model(precision,gemm_layer)

  return results

In [60]:
def conv(precision,conv_layer):   # Roofline Model Baseline
  execution_time = 0

  # Here is some more filler code

  results = roofline_model(precision,conv_layer)

  return results

In [61]:
def im2col(conv_layer):
  # im2col is used to convert convolution into a GEMM
  # Each convolution is unrolled. For Ni input channels that are Kx x Kn and Nn input filters, this would mean our gemm will be of dimensions (m,k)x(k,n) where m = Nn, k = Ni*Kx*Ky, and n = number_output_values_per_channel.
  gemm_layer = GemmLayer(conv_layer.name+"_gemm", conv_layer.num_filters, conv_layer.filter_shape[0]*conv_layer.filter_shape[1]*conv_layer.input_shape[0], (conv_layer.input_shape[1]-conv_layer.filter_shape[0]+1)*(conv_layer.input_shape[2]-conv_layer.filter_shape[1]+1), conv_layer.batch_size)
  return gemm_layer

In [62]:
def find_tiled_times(precision,gemm_layer):
  total_time = 0
  tile_dim = 1    #Tiles are assumed to be square.
  row_col_size = gemm_layer.k

  if precision == "FP16":
    precision_multiplier  = 2
  elif precision == "FP32":
    precision_multiplier = 4
  elif precision == "FP64":
    precision_multiplier = 8
  else: #"INT8"
    precision_multiplier = 1

  total_size = (gemm_layer.m*gemm_layer.n+gemm_layer.k*(gemm_layer.m+gemm_layer.n))*precision_multiplier

  if (total_size <= Shared_Memory_Size):
    bound_str = "memory"
    memory_bound = find_memory_bound_time(precision, gemm_layer)
    total_time = memory_bound
    compute_bound = find_compute_bound_time(precision, gemm_layer)
    if (compute_bound > total_time):
      bound_str = "compute"
      total_time = compute_bound
    buffer_bound = find_buffer_bandwidth_bound_time(precision, gemm_layer)
    if (buffer_bound > total_time):
      bound_str = "buffer"
      total_time = buffer_bound
  else:
    #Find tile size. Start with an overestimate
    tile_dim = math.floor(Shared_Memory_Size/(2*row_col_size))   # This ignores the output values, so we need a correction.
    tile_size = tile_dim*(tile_dim+2*row_col_size)*precision_multiplier
    while tile_size > Shared_Memory_Size:
      tile_dim = tile_dim - 1
      tile_size = tile_dim*(tile_dim+2*row_col_size)*precision_multiplier

    #Iterate through tiles. Note that we assume the tiles iterate along the n dimension first and all weights only need to fully change at the end of each row.
    #Number of full tiles:
    n_tiles = math.floor(gemm_layer.n/tile_dim)
    m_tiles = math.floor(gemm_layer.m/tile_dim)
    buffer_bound = time_to_load_from_L2_to_L1(precision,tile_size*n_tiles*m_tiles+row_col_size*n_tiles*(m_tiles+1)*precision_multiplier)
    #Partial tiles. These tiles will require less loaded memory and fewer operations to execute.
    leftover_n = gemm_layer.n - tile_dim*n_tiles
    leftover_m = gemm_layer.m - tile_dim*m_tiles
    buffer_bound = buffer_bound + time_to_load_from_L2_to_L1(precision,(leftover_m+(leftover_n*row_col_size)*(m_tiles))*precision_multiplier)
    # Add Compute Bound Time
    compute_bound = find_compute_bound_time(precision, gemm_layer)
    memory_bound = find_memory_bound_time(precision, gemm_layer)
    total_time = buffer_bound*gemm_layer.batch_size
    bound_str = "buffer"
    if (memory_bound > total_time):
      bound_str = "memory"
      total_time = memory_bound
    if (compute_bound > total_time):
      bound_str = "compute"
      total_time = compute_bound

  return (total_time+STARTUP_TIME, bound_str)

## Test Driver Code

In [69]:
precision = "FP16"

if verifyPrecisionValue(precision):
  # # Loop through parameters and layer sizes for gemm here
  # print("===============")
  # print("GEMM Results")
  # print(f"===============\n")
  # total = 0
  # for i in range(len(gemm_layers)):  # Placeholder loop
  #   res = gemm(precision,gemm_layers[i])
  #   total += res[0]
  #   print(gemm_layers[i].name + " Execution Time: " + str(res[0]) + "\n\tbounded by: " + res[1]+ "\n")

  # print("\nTotal execution time: {}".format(total))

  # print("===============")
  # print(f"===============\n\n\n")

  # Loop through parameters and layer sizes for conv here
  print("===============")
  print("Conv Results")
  print(f"===============\n")
  total = 0
  for i in range(len(conv_layers)):  # Placeholder loop
    res = conv(precision,conv_layers[i])
    total += res[0]
    print(conv_layers[i].name + " Execution Time: " + str(res[0]) + "\n\tbounded by: " + res[1] + "\n")

  print("\nTotal execution time: {}".format(total))

  print("===============")
  print(f"===============\n\n\n")
  
  print("===============")
  print("im2col Results for Conv as GEMM")
  print(f"===============\n")
  total = 0
  for i in range(len(conv_layers)):  # Placeholder loop
    im2col_layer = im2col(conv_layers[i])
    res = find_tiled_times(precision,im2col_layer)
    total += res[0]
    print(im2col_layer.name + " Execution Time: " + str(res[0]) + "\n\tbounded by: " + res[1] + "\n")

  print("\nTotal execution time: {}".format(total))

  print("===============")
  print("===============")
else:
  print("Incorrect precision value: "+precision+" please enter a string from the following list: FP16, FP32, FP64, INT8")

Conv Results

Compute: 0.00013165256347826086
Data size is load 3248128 and output 3154176
Memory: 0.00010213276301587301
Data size is load 3248128 and output 3154176
Buffer: 2.1484241610738256e-06
Conv_1_b1 Execution Time: 0.00013286856347826087
	bounded by: compute

Compute: 0.0002633051269565217
Data size is load 6459392 and output 6308352
Memory: 0.00020317149317460316
Data size is load 6459392 and output 6308352
Buffer: 4.284477852348994e-06
Conv_1_b2 Execution Time: 0.0002645211269565217
	bounded by: compute

Compute: 0.0005266102539130434
Data size is load 12881920 and output 12616704
Memory: 0.0004052489534920635
Data size is load 12881920 and output 12616704
Buffer: 8.556585234899329e-06
Conv_1_b4 Execution Time: 0.0005278262539130434
	bounded by: compute

Compute: 0.0010532205078260868
Data size is load 25726976 and output 25233408
Memory: 0.0008094038741269841
Data size is load 25726976 and output 25233408
Buffer: 1.71008e-05
Conv_1_b8 Execution Time: 0.001054436507826087
	b